In [1]:
import pandas as pd
from scipy.stats import binom_test


In [2]:
pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

rel = pd.read_csv('../data/family_summaries.tsv', sep='\t')
rel = rel.set_index('subject', drop=False)

In [3]:
ase = pd.read_csv('ase_cadd_annotated.tsv', sep='\t')

# inherited or de novo

In [19]:
affected_child = ['SG001', 'SG011', 'SG021', 'SG026', 'SG155', 'SG041', 'SG042', 'SG037', 'SG148', 'SG022']
carrier_parents = ['SG006', 'SG025', 'SG069', 'SG040', 'SG044', 'SG151']
noncarrier_parents = ['SG007', 'SG024', 'SG027', 'SG039', 'SG043','SG152']

df = pd.DataFrame(index = rel.index)
df['subject'] = df.index.to_series()
df['family'] = pheno.loc[df.subject, 'family'].to_list()

df['ase_oe'] = 0

for child in pheno.index:    
    if child not in rel.index:
        continue
    c_ase = ase[(ase.subject == child) & (ase.cadd_greater_than_25 == 'X')].ensembl.to_list()
    
    df.at[child, 'ase_oe'] = len(c_ase)
    
    cp = rel.at[child, 'carrier_parent']
    ncp = rel.at[child, 'non_carrier_parent']
    cp_ase = ase[(ase.subject == cp) & (ase.cadd_greater_than_25 == 'X')].ensembl.to_list()
    ncp_ase = ase[(ase.subject == ncp) & (ase.cadd_greater_than_25 == 'X')].ensembl.to_list()
    
#     print(c_ase, cp_ase, ncp_ase)
    denovo = []
    icp = []
    incp = []
    iboth = []
    for gene in c_ase:
        if (gene in cp_ase) and (gene in ncp_ase):
            iboth.append(gene)
        elif gene in cp_ase:
            icp.append(gene)
        elif gene in ncp_ase:
            incp.append(gene)
        else:
            denovo.append(gene)
            
    df.at[child, 'ase_oe_de_novo'] = len(denovo)
    df.at[child, 'ase_oe_inh_both'] = len(iboth)
    df.at[child, 'ase_oe_inh_cp'] = len(icp)
    df.at[child, 'ase_oe_inh_ncp'] = len(incp)
    
df = df.drop('SG011')

In [21]:
df.shape

(17, 7)

In [22]:

df.to_csv('ase_summary_deletrious_variant.tsv', sep='\t', index=False)

In [23]:
df


,subject,family,ase_oe,ase_oe_de_novo,ase_oe_inh_both,ase_oe_inh_cp,ase_oe_inh_ncp
subject,,,,,,,
SG001,SG001,PSU001,1,1.0,0.0,0.0,0.0
SG006,SG006,PSU001,2,2.0,0.0,0.0,0.0
SG021,SG021,PSU004,0,0.0,0.0,0.0,0.0
SG022,SG022,PSU004,0,0.0,0.0,0.0,0.0
SG023,SG023,PSU004,1,0.0,0.0,1.0,0.0
SG155,SG155,PSU005,0,0.0,0.0,0.0,0.0
SG069,SG069,PSU005,0,0.0,0.0,0.0,0.0
SG026,SG026,PSU005,0,0.0,0.0,0.0,0.0
SG041,SG041,PSU007,2,2.0,0.0,0.0,0.0
